<a href="https://colab.research.google.com/github/testgithubprecious/Ml_projects/blob/main/LIME_SHAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Install dependencies if not already:
# pip install scikit-learn lime shap matplotlib

import numpy as np
import pandas as pd
import shap
import lime
import lime.lime_tabular
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# -----------------------
# Load dataset
# -----------------------
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target
feature_names = data.feature_names

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# -----------------------
# Train a black-box model
# -----------------------
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# -----------------------
# Pick multiple instances to explain
# -----------------------
indices_to_explain = [5, 10, 15]  # example instances

# -----------------------
# LIME Explanation
# -----------------------
explainer_lime = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=feature_names,
    class_names=data.target_names,
    mode="classification"
)

for idx in indices_to_explain:
    instance = X_test.iloc[idx:idx+1].values
    exp = explainer_lime.explain_instance(instance[0], model.predict_proba, num_features=5)
    print(f"\n🧠 LIME Explanation for instance {idx}:")
    # In scripts, render with matplotlib
    fig = exp.as_pyplot_figure()
    plt.show()

# -----------------------
# SHAP Explanation
# -----------------------
explainer_shap = shap.TreeExplainer(model)
shap_values = explainer_shap.shap_values(X_test)

# Summary plot (global feature importance)
print("\n🧠 SHAP Explanation (summary plot):")
shap.summary_plot(shap_values[1], X_test, feature_names=feature_names, show=True)

# Force plot for individual instances
for idx in indices_to_explain:
    print(f"\n🧠 SHAP Force Plot for instance {idx}:")
    shap.force_plot(
        explainer_shap.expected_value[1],
        shap_values[1][idx],
        X_test.iloc[idx],
        matplotlib=True
    )